In [1]:
%matplotlib inline

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Tensorflow Limit GPU usage
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [4]:
# Dimension constants
im_ht = 4
im_wt = 4

kr_ht = 2
kr_wt = 2

no_ch = 3
no_im = 1
no_kr = 1

y_ht = im_ht - kr_ht + 1
y_wt = im_wt - kr_wt + 1

no_ph = y_ht*y_wt

In [ ]:
# creating 1x4x4x3 image #[batch_no, in_height, in_width, in_channels]

In [5]:
b0_chr = np.arange(im_ht * im_wt, dtype="float32")
b0_chr

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15.], dtype=float32)

In [6]:
b0_chg = b0_chr*10 + 5
b0_chg

array([  5.,  15.,  25.,  35.,  45.,  55.,  65.,  75.,  85.,  95., 105.,
       115., 125., 135., 145., 155.], dtype=float32)

In [7]:
b0_chb = b0_chr*10 + 7
b0_chb

array([  7.,  17.,  27.,  37.,  47.,  57.,  67.,  77.,  87.,  97., 107.,
       117., 127., 137., 147., 157.], dtype=float32)

In [8]:
image = np.stack((b0_chr, b0_chg, b0_chb), axis = 1)
image = tf.constant(image)

In [9]:
image = tf.reshape(image, (no_im, im_ht, im_wt, no_ch))
image

<tf.Tensor: shape=(1, 4, 4, 3), dtype=float32, numpy=
array([[[[  0.,   5.,   7.],
         [  1.,  15.,  17.],
         [  2.,  25.,  27.],
         [  3.,  35.,  37.]],

        [[  4.,  45.,  47.],
         [  5.,  55.,  57.],
         [  6.,  65.,  67.],
         [  7.,  75.,  77.]],

        [[  8.,  85.,  87.],
         [  9.,  95.,  97.],
         [ 10., 105., 107.],
         [ 11., 115., 117.]],

        [[ 12., 125., 127.],
         [ 13., 135., 137.],
         [ 14., 145., 147.],
         [ 15., 155., 157.]]]], dtype=float32)>

In [10]:
image.shape

TensorShape([1, 4, 4, 3])

In [ ]:
# creating 2x2x3x1 kernel # [filter_height, filter_width, in_channels, out_channels]

In [11]:
kr_r = np.array([[5,5],[6,6]], dtype='float32')
kr_r

array([[5., 5.],
       [6., 6.]], dtype=float32)

In [12]:
kr_g = kr_r*10+1
kr_g

array([[51., 51.],
       [61., 61.]], dtype=float32)

In [13]:
kr_b = -kr_r
kr_b

array([[-5., -5.],
       [-6., -6.]], dtype=float32)

In [14]:
kr = tf.constant(np.stack([kr_r, kr_g, kr_b], axis=2))

In [15]:
kr = tf.reshape(kr, (kr_ht, kr_wt, no_ch, no_kr))
kr

<tf.Tensor: shape=(2, 2, 3, 1), dtype=float32, numpy=
array([[[[ 5.],
         [51.],
         [-5.]],

        [[ 5.],
         [51.],
         [-5.]]],


       [[[ 6.],
         [61.],
         [-6.]],

        [[ 6.],
         [61.],
         [-6.]]]], dtype=float32)>

In [ ]:
# extracting image patches

In [16]:
patches = tf.image.extract_patches(images=image,
                                     sizes=[1, kr_ht, kr_wt, 1],
                                     strides=[1, 1, 1, 1],
                                     rates=[1, 1, 1, 1],
                                     padding='VALID')

ph_len = patches.shape[-1]
patches

<tf.Tensor: shape=(1, 3, 3, 12), dtype=float32, numpy=
array([[[[  0.,   5.,   7.,   1.,  15.,  17.,   4.,  45.,  47.,   5.,
           55.,  57.],
         [  1.,  15.,  17.,   2.,  25.,  27.,   5.,  55.,  57.,   6.,
           65.,  67.],
         [  2.,  25.,  27.,   3.,  35.,  37.,   6.,  65.,  67.,   7.,
           75.,  77.]],

        [[  4.,  45.,  47.,   5.,  55.,  57.,   8.,  85.,  87.,   9.,
           95.,  97.],
         [  5.,  55.,  57.,   6.,  65.,  67.,   9.,  95.,  97.,  10.,
          105., 107.],
         [  6.,  65.,  67.,   7.,  75.,  77.,  10., 105., 107.,  11.,
          115., 117.]],

        [[  8.,  85.,  87.,   9.,  95.,  97.,  12., 125., 127.,  13.,
          135., 137.],
         [  9.,  95.,  97.,  10., 105., 107.,  13., 135., 137.,  14.,
          145., 147.],
         [ 10., 105., 107.,  11., 115., 117.,  14., 145., 147.,  15.,
          155., 157.]]]], dtype=float32)>

In [ ]:
# convolution without reordering image into different channels

In [17]:
kr_2 = tf.reshape(kr, (no_kr, kr_ht*kr_wt*no_ch))
kr_2

<tf.Tensor: shape=(1, 12), dtype=float32, numpy=
array([[ 5., 51., -5.,  5., 51., -5.,  6., 61., -6.,  6., 61., -6.]],
      dtype=float32)>

In [18]:
patches_2 = tf.reshape(patches, (no_ph,-1))
patches_2 = tf.transpose(patches_2, (1,0))
patches_2

<tf.Tensor: shape=(12, 9), dtype=float32, numpy=
array([[  0.,   1.,   2.,   4.,   5.,   6.,   8.,   9.,  10.],
       [  5.,  15.,  25.,  45.,  55.,  65.,  85.,  95., 105.],
       [  7.,  17.,  27.,  47.,  57.,  67.,  87.,  97., 107.],
       [  1.,   2.,   3.,   5.,   6.,   7.,   9.,  10.,  11.],
       [ 15.,  25.,  35.,  55.,  65.,  75.,  95., 105., 115.],
       [ 17.,  27.,  37.,  57.,  67.,  77.,  97., 107., 117.],
       [  4.,   5.,   6.,   8.,   9.,  10.,  12.,  13.,  14.],
       [ 45.,  55.,  65.,  85.,  95., 105., 125., 135., 145.],
       [ 47.,  57.,  67.,  87.,  97., 107., 127., 137., 147.],
       [  5.,   6.,   7.,   9.,  10.,  11.,  13.,  14.,  15.],
       [ 55.,  65.,  75.,  95., 105., 115., 135., 145., 155.],
       [ 57.,  67.,  77.,  97., 107., 117., 137., 147., 157.]],
      dtype=float32)>

In [19]:
conv_out_2= kr_2@patches_2
conv_out_2 = tf.reshape(conv_out_2, (y_ht,y_wt))
conv_out_2

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[ 6435.,  8477., 10519.],
       [14603., 16645., 18687.],
       [22771., 24813., 26855.]], dtype=float32)>

In [ ]:
# convolution after reordering image and kernels into RGB sections

In [ ]:
# reordering patches in images

In [20]:
ip_flat = tf.reshape(patches, (no_ph, ph_len))
ip_flat

<tf.Tensor: shape=(9, 12), dtype=float32, numpy=
array([[  0.,   5.,   7.,   1.,  15.,  17.,   4.,  45.,  47.,   5.,  55.,
         57.],
       [  1.,  15.,  17.,   2.,  25.,  27.,   5.,  55.,  57.,   6.,  65.,
         67.],
       [  2.,  25.,  27.,   3.,  35.,  37.,   6.,  65.,  67.,   7.,  75.,
         77.],
       [  4.,  45.,  47.,   5.,  55.,  57.,   8.,  85.,  87.,   9.,  95.,
         97.],
       [  5.,  55.,  57.,   6.,  65.,  67.,   9.,  95.,  97.,  10., 105.,
        107.],
       [  6.,  65.,  67.,   7.,  75.,  77.,  10., 105., 107.,  11., 115.,
        117.],
       [  8.,  85.,  87.,   9.,  95.,  97.,  12., 125., 127.,  13., 135.,
        137.],
       [  9.,  95.,  97.,  10., 105., 107.,  13., 135., 137.,  14., 145.,
        147.],
       [ 10., 105., 107.,  11., 115., 117.,  14., 145., 147.,  15., 155.,
        157.]], dtype=float32)>

In [21]:
ip_flat = tf.reshape(ip_flat, (no_ph, kr_ht, kr_wt, no_ch))
ip_flat

<tf.Tensor: shape=(9, 2, 2, 3), dtype=float32, numpy=
array([[[[  0.,   5.,   7.],
         [  1.,  15.,  17.]],

        [[  4.,  45.,  47.],
         [  5.,  55.,  57.]]],


       [[[  1.,  15.,  17.],
         [  2.,  25.,  27.]],

        [[  5.,  55.,  57.],
         [  6.,  65.,  67.]]],


       [[[  2.,  25.,  27.],
         [  3.,  35.,  37.]],

        [[  6.,  65.,  67.],
         [  7.,  75.,  77.]]],


       [[[  4.,  45.,  47.],
         [  5.,  55.,  57.]],

        [[  8.,  85.,  87.],
         [  9.,  95.,  97.]]],


       [[[  5.,  55.,  57.],
         [  6.,  65.,  67.]],

        [[  9.,  95.,  97.],
         [ 10., 105., 107.]]],


       [[[  6.,  65.,  67.],
         [  7.,  75.,  77.]],

        [[ 10., 105., 107.],
         [ 11., 115., 117.]]],


       [[[  8.,  85.,  87.],
         [  9.,  95.,  97.]],

        [[ 12., 125., 127.],
         [ 13., 135., 137.]]],


       [[[  9.,  95.,  97.],
         [ 10., 105., 107.]],

        [[ 13., 135., 137.],
   

In [22]:
ip_flat = tf.transpose(ip_flat, perm=(0,3,1,2))
ip_flat

<tf.Tensor: shape=(9, 3, 2, 2), dtype=float32, numpy=
array([[[[  0.,   1.],
         [  4.,   5.]],

        [[  5.,  15.],
         [ 45.,  55.]],

        [[  7.,  17.],
         [ 47.,  57.]]],


       [[[  1.,   2.],
         [  5.,   6.]],

        [[ 15.,  25.],
         [ 55.,  65.]],

        [[ 17.,  27.],
         [ 57.,  67.]]],


       [[[  2.,   3.],
         [  6.,   7.]],

        [[ 25.,  35.],
         [ 65.,  75.]],

        [[ 27.,  37.],
         [ 67.,  77.]]],


       [[[  4.,   5.],
         [  8.,   9.]],

        [[ 45.,  55.],
         [ 85.,  95.]],

        [[ 47.,  57.],
         [ 87.,  97.]]],


       [[[  5.,   6.],
         [  9.,  10.]],

        [[ 55.,  65.],
         [ 95., 105.]],

        [[ 57.,  67.],
         [ 97., 107.]]],


       [[[  6.,   7.],
         [ 10.,  11.]],

        [[ 65.,  75.],
         [105., 115.]],

        [[ 67.,  77.],
         [107., 117.]]],


       [[[  8.,   9.],
         [ 12.,  13.]],

        [[ 85.,  95.],

In [23]:
ip_flat = tf.reshape(ip_flat, (no_ph,-1))
ip_flat

<tf.Tensor: shape=(9, 12), dtype=float32, numpy=
array([[  0.,   1.,   4.,   5.,   5.,  15.,  45.,  55.,   7.,  17.,  47.,
         57.],
       [  1.,   2.,   5.,   6.,  15.,  25.,  55.,  65.,  17.,  27.,  57.,
         67.],
       [  2.,   3.,   6.,   7.,  25.,  35.,  65.,  75.,  27.,  37.,  67.,
         77.],
       [  4.,   5.,   8.,   9.,  45.,  55.,  85.,  95.,  47.,  57.,  87.,
         97.],
       [  5.,   6.,   9.,  10.,  55.,  65.,  95., 105.,  57.,  67.,  97.,
        107.],
       [  6.,   7.,  10.,  11.,  65.,  75., 105., 115.,  67.,  77., 107.,
        117.],
       [  8.,   9.,  12.,  13.,  85.,  95., 125., 135.,  87.,  97., 127.,
        137.],
       [  9.,  10.,  13.,  14.,  95., 105., 135., 145.,  97., 107., 137.,
        147.],
       [ 10.,  11.,  14.,  15., 105., 115., 145., 155., 107., 117., 147.,
        157.]], dtype=float32)>

In [24]:
ip_flat = tf.transpose(ip_flat, [1,0])
ip_flat

<tf.Tensor: shape=(12, 9), dtype=float32, numpy=
array([[  0.,   1.,   2.,   4.,   5.,   6.,   8.,   9.,  10.],
       [  1.,   2.,   3.,   5.,   6.,   7.,   9.,  10.,  11.],
       [  4.,   5.,   6.,   8.,   9.,  10.,  12.,  13.,  14.],
       [  5.,   6.,   7.,   9.,  10.,  11.,  13.,  14.,  15.],
       [  5.,  15.,  25.,  45.,  55.,  65.,  85.,  95., 105.],
       [ 15.,  25.,  35.,  55.,  65.,  75.,  95., 105., 115.],
       [ 45.,  55.,  65.,  85.,  95., 105., 125., 135., 145.],
       [ 55.,  65.,  75.,  95., 105., 115., 135., 145., 155.],
       [  7.,  17.,  27.,  47.,  57.,  67.,  87.,  97., 107.],
       [ 17.,  27.,  37.,  57.,  67.,  77.,  97., 107., 117.],
       [ 47.,  57.,  67.,  87.,  97., 107., 127., 137., 147.],
       [ 57.,  67.,  77.,  97., 107., 117., 137., 147., 157.]],
      dtype=float32)>

In [ ]:
# reordering kernels into rgb channels

In [25]:
kr

<tf.Tensor: shape=(2, 2, 3, 1), dtype=float32, numpy=
array([[[[ 5.],
         [51.],
         [-5.]],

        [[ 5.],
         [51.],
         [-5.]]],


       [[[ 6.],
         [61.],
         [-6.]],

        [[ 6.],
         [61.],
         [-6.]]]], dtype=float32)>

In [35]:
kr_flat = tf.transpose(kr, perm=(2,0,1,3))
kr_flat

<tf.Tensor: shape=(3, 2, 2, 1), dtype=float32, numpy=
array([[[[ 5.],
         [ 5.]],

        [[ 6.],
         [ 6.]]],


       [[[51.],
         [51.]],

        [[61.],
         [61.]]],


       [[[-5.],
         [-5.]],

        [[-6.],
         [-6.]]]], dtype=float32)>

In [37]:
kr_flat = tf.reshape(kr_flat, [no_kr,-1])
kr_flat

<tf.Tensor: shape=(1, 12), dtype=float32, numpy=
array([[ 5.,  5.,  6.,  6., 51., 51., 61., 61., -5., -5., -6., -6.]],
      dtype=float32)>

In [38]:
kr_flat.shape

TensorShape([1, 12])

In [39]:
ip_flat.shape

TensorShape([12, 9])

In [40]:
conv_out = kr_flat@ip_flat
conv_out = tf.reshape(conv_out, (y_ht,y_wt))
conv_out

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[ 6435.,  8477., 10519.],
       [14603., 16645., 18687.],
       [22771., 24813., 26855.]], dtype=float32)>

In [ ]:
# using tensorflow built-in method

In [41]:
image.shape #[batch_no, in_height, in_width, in_channels]

TensorShape([1, 4, 4, 3])

In [32]:
kr.shape # [filter_height, filter_width, in_channels, out_channels]

TensorShape([2, 2, 3, 1])

In [43]:
tf_conv_out = tf.nn.conv2d(input=image,
                            filters=kr,
                            strides=[1,1,1,1],
                            padding="VALID",
                            data_format='NHWC',
                            dilations=None,
                            name=None
                        )

In [44]:
tf_conv_out

<tf.Tensor: shape=(1, 3, 3, 1), dtype=float32, numpy=
array([[[[ 6435.],
         [ 8477.],
         [10519.]],

        [[14603.],
         [16645.],
         [18687.]],

        [[22771.],
         [24813.],
         [26855.]]]], dtype=float32)>

In [ ]:
sdfd

In [ ]:
patch0 = patches[0,0,0,:]

In [ ]:
patch0

In [ ]:
# convert to rgb patch
y = tf.reshape(patch0, (kr_ht, kr_wt, no_ch))

In [ ]:
# rearrange the image channel wise
y = tf.transpose(y, (2,0,1))
y

In [ ]:
# flatten patch
y = tf.reshape(y, (kr_ht*kr_wt*no_ch,-1))

In [ ]:
y